In [16]:
import os
import yake
import nltk

In [17]:
from collections import defaultdict
import sys
import spacy
from spacy.lang.fr.examples import sentences

In [18]:
nlp = spacy.load('fr_core_news_md')

In [19]:
# Instantier l'extracteur de mots clés
kw_extractor = yake.KeywordExtractor(lan="fr", top=50)
kw_extractor

In [20]:
year = 1966

data_path = '../data'

txt_path = '../data/txt'

txts = [f for f in os.listdir(txt_path) if os.path.isfile(os.path.join(txt_path, f)) and str(year) in f]

len(txts)

100

In [21]:
# Les dix premiers fichiers
txts[:10]

['KB_JB838_1966-08-30_01-00011.txt',
 'KB_JB838_1966-07-12_01-00017.txt',
 'KB_JB838_1966-02-26_01-00002.txt',
 'KB_JB838_1966-01-29_01-00013.txt',
 'KB_JB838_1966-02-16_01-00006.txt',
 'KB_JB838_1966-06-18_01-00028.txt',
 'KB_JB838_1966-11-20_01-00023.txt',
 'KB_JB838_1966-06-08_01-00026.txt',
 'KB_JB838_1966-02-10_01-00001.txt',
 'KB_JB838_1966-11-26_01-00032.txt']

In [22]:
# Choisir un fichier
this_file = txts[12]
this_file

'KB_JB838_1966-12-08_01-00012.txt'

In [23]:
# Récupérer le texte du fichier
text = open(os.path.join(txt_path, this_file), 'r', encoding='utf-8').read()
text[:500]

"12 . LE SOIR Jeudi, face à Honved Budapest, en match-aller de 8 me de finale de la Coupe d'Europe Le Racing de Malines doit limiter les dégâts T JEUDI 8 DECEMBRE 1966 - - - Pas plus étoffé que les dernières années Exempté du premier tour éliminatoire, le Racing de Malines débutera jeudi après midi (15 h 45) à Budapest en Coupe d’Europe des clubs champions. Il y rencontrera Honved Budapest en match-aller de huitièmes de linale, la seconde manche se disputant, le 15 novembre, à l’Arena de Deurne. "

In [24]:
# Extraire les mots clés de ce texte
keywords = kw_extractor.extract_keywords(text)
keywords

[('pts', 0.0012962724451267673),
 ('Belgique', 0.009190354594685627),
 ('Red Sox', 0.009241816933868826),
 ('Prix', 0.01207567444234317),
 ('Van', 0.01249401031510284),
 ('tour', 0.016081677350206543),
 ('SOIR', 0.016341542609950303),
 ('Honved Budapest', 0.016433876463239974),
 ('Racing de Malines', 0.017205404804297848),
 ('Racing', 0.017788106016153743),
 ('Mme', 0.018033629353808523),
 ('Malines', 0.018188728109897717),
 ('Coupe', 0.018398207086884156),
 ('tours', 0.020102096687758177),
 ('Joël Robert', 0.021397150648077366),
 ('Para', 0.02531540501086204),
 ('Van Hoof', 0.03170707030351907),
 ('Grand Prix', 0.032036924411983456),
 ('Coupe d’Europe', 0.03250368183855206),
 ('Red', 0.03253647793562752),
 ('Sox', 0.033061231536647126),
 ('Budapest', 0.0345797580283415),
 ('club', 0.03608626555598461),
 ('Honved', 0.03640261180384409),
 ('tournoi', 0.03684704810112896),
 ('Métissé', 0.03908772131632472),
 ('mardi soir', 0.03931823894756685),
 ('France', 0.039728566581680044),
 ('Rober

In [25]:
# Ne garder que les bigrammes
kept = []
for kw, score in keywords:
    words = kw.split()
    if len(words) == 2:
        kept.append(kw)
kept

['Red Sox',
 'Honved Budapest',
 'Joël Robert',
 'Van Hoof',
 'Grand Prix',
 'Coupe d’Europe',
 'mardi soir',
 'Racing White']

In [26]:
# Stocker le contenu de ces fichiers dans une liste
content_list = []
for txt in txts:
    with open(os.path.join(txt_path, txt), 'r') as f:
        content_list.append(f.read())

In [27]:
# Ecrire tout le contenu dans un fichier temporaire
temp_path = '../data/tmp'
if not os.path.exists(temp_path):
    os.mkdir(temp_path)
with open(os.path.join(temp_path, f'{year}.txt'), 'w') as f:
    f.write(' '.join(content_list))

In [28]:
# Nettoyer le texte avec fonction de nettoyage

def clean_text(year, folder=None):
    if folder is None:
        input_path = f"{year}.txt"
        output_path = f"{year}_clean.txt"
    else:
        input_path = f"{folder}/{year}.txt"
        output_path = f"{folder}/{year}_clean.txt"
    output = open(output_path, "w", encoding='utf-8')
    with open(input_path, encoding='utf-8') as f:
        text = f.read()
        words = nltk.wordpunct_tokenize(text)
        kept = [w.lower() for w in words if len(w) > 2 and w.isalpha() and w.lower() not in sw]
        kept_string = " ".join(kept)
        output.write(kept_string)
    return f'Output has been written in {output_path}!'

In [29]:
# Charger le texte
n=100000
text = open(os.path.join(temp_path, f'{year}_clean.txt'), encoding='utf-8').read()[:n]

In [30]:
%%time
# Traiter le texte

doc = nlp(text)

CPU times: user 5.71 s, sys: 479 ms, total: 6.19 s
Wall time: 7.87 s


In [31]:
# Compter les entités
people = defaultdict(int)
for ent in doc.ents:
    if ent.label_ == "PER" and len(ent.text) > 3:
        people[ent.text] += 1

In [32]:
# Trier et imprimer

sorted_people = sorted(people.items(), key=lambda kv: kv[1], reverse=True)

for person, freq in sorted_people[:50]:
    print(f"{person} apparait {freq} fois dans le corpus")

siniavski daniel apparait 4 fois dans le corpus
siniavski apparait 4 fois dans le corpus
scott apparait 4 fois dans le corpus
daniel apparait 3 fois dans le corpus
borel delsart apparait 2 fois dans le corpus
borel apparait 2 fois dans le corpus
armstrong apparait 2 fois dans le corpus
roger apparait 2 fois dans le corpus
itional sau plu can apparait 1 fois dans le corpus
précé olr apparait 1 fois dans le corpus
iclaré apparait 1 fois dans le corpus
devrlo apparait 1 fois dans le corpus
irge frali apparait 1 fois dans le corpus
jean pierre vaste voiture apparait 1 fois dans le corpus
ille falalsi apparait 1 fois dans le corpus
hom libre apparait 1 fois dans le corpus
salüt pat apparait 1 fois dans le corpus
leibnitz apparait 1 fois dans le corpus
lortmiolsl hayna tilianus ncourt lajeur apparait 1 fois dans le corpus
guy îud iste fris cur ficiers internationale wels apparait 1 fois dans le corpus
tim présenlant apparait 1 fois dans le corpus
ihu plan apparait 1 fois dans le corpus
gousi